In [1]:
import re
import pandas as pd
import numpy as np
from plotnine import *
from matplotlib import pyplot as plt
from datetime import datetime, timedelta
from tqdm import tqdm
import nltk
from nltk.tokenize import WhitespaceTokenizer
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import LatentDirichletAllocation
from sklearn.datasets import make_multilabel_classification
from sklearn.feature_extraction.text import CountVectorizer
import pyLDAvis
import pyLDAvis.sklearn
# nltk.download()

In [2]:
tweets = pd.read_csv('./valorant_tweets.csv')
tweets

,tweet,date,Agent,Role
0,and now i want brimstone in my garden\ni want ...,2021-09-29 22:30:50,Brimstone,Controller
1,why half of the brimstone players are literall...,2021-09-29 22:14:44,Brimstone,Controller
2,@DavidLevy101 @Meloniemaher1 That fucker gets ...,2021-09-29 22:07:50,Brimstone,Controller
3,They also did these cute lil' ghosts! \n\nRobo...,2021-09-29 22:01:21,Brimstone,Controller
4,@AuthorDelaneyW @mis_cue @C_Stroop That is not...,2021-09-29 21:57:18,Brimstone,Controller
...,...,...,...,...
15497,I'm really confused as to why they did what th...,2021-09-29 07:17:43,Kayo,Initiator
15498,@ChidgR @maxblake44 @Foxtel @kayosports @arara...,2021-09-29 07:15:34,Kayo,Initiator
15499,May interested ba \n\nNCT STICKY VER SEALED\n\...,2021-09-29 07:13:49,Kayo,Initiator
15500,@BrokeOnTwice Shout out to all Communities her...,2021-09-29 07:12:59,Kayo,Initiator


In [3]:
tweets.describe()

,tweet,date,Agent,Role
count,15502,15502,15502,15502
unique,15171,13728,16,4
top,"and now i found brimstone in my garden,\ni fou...",2021-09-29 21:45:00,Brimstone,Duelist
freq,13,23,1000,4502


In [4]:
agents = []
loweragents = []
for agent in tweets.loc[:,'Agent'].values:
    if agent not in agents:
        agents.append(agent)
        loweragents.append(agent.lower())
print(agents)

['Brimstone', 'Phoenix', 'Sage', 'Sova', 'Viper', 'Cypher', 'Reyna', 'Killjoy', 'Breach', 'Omen', 'Jett', 'Raze', 'Skye', 'Yoru', 'Astra', 'Kayo']


In [5]:
role = {}
for idx, row in tweets.iterrows():
    if row['Role'] not in role:
        role[row['Role']] = [row['Agent']]
    elif row['Role'] in role and row['Agent'] not in role[row['Role']]:
        role[row['Role']].append(row['Agent'])

In [6]:
tweets

,tweet,date,Agent,Role
0,and now i want brimstone in my garden\ni want ...,2021-09-29 22:30:50,Brimstone,Controller
1,why half of the brimstone players are literall...,2021-09-29 22:14:44,Brimstone,Controller
2,@DavidLevy101 @Meloniemaher1 That fucker gets ...,2021-09-29 22:07:50,Brimstone,Controller
3,They also did these cute lil' ghosts! \n\nRobo...,2021-09-29 22:01:21,Brimstone,Controller
4,@AuthorDelaneyW @mis_cue @C_Stroop That is not...,2021-09-29 21:57:18,Brimstone,Controller
...,...,...,...,...
15497,I'm really confused as to why they did what th...,2021-09-29 07:17:43,Kayo,Initiator
15498,@ChidgR @maxblake44 @Foxtel @kayosports @arara...,2021-09-29 07:15:34,Kayo,Initiator
15499,May interested ba \n\nNCT STICKY VER SEALED\n\...,2021-09-29 07:13:49,Kayo,Initiator
15500,@BrokeOnTwice Shout out to all Communities her...,2021-09-29 07:12:59,Kayo,Initiator


In [7]:
wst = WhitespaceTokenizer()
ps = PorterStemmer()
stop_words = stopwords.words('english')
maxwords = 0
maxindex = 0
print(stop_words)
def cleantweet(tweet):
    filteredtweets = []
    for word in wst.tokenize(tweet.lower()):
        if word not in stop_words and '@' not in word: #remove stopwords and @s
            word = re.sub(r"^(https|http):\/\/t.co\/.+|\W",'',word) #remove links
            if word != "": # didnt remove word in the if statement before because regex might replace as ''
                    if word not in loweragents:
                        word = ps.stem(word) #get word stem
                    filteredtweets.append(word)
    return filteredtweets
listofwords=[]
listoftweets=[]
with tqdm(total = len(tweets)) as pbar:
    for i in range(len(tweets)):
        pbar.update(1)
        listofwords.append(cleantweet(tweets.loc[i,'tweet']))
for i in range(len(listofwords)):
    listoftweets.append(" ".join(listofwords[i]))
tweets['tl'] = listofwords
tweets['tweetjoin'] = listoftweets
tweets

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', '

100%|██████████████████████████████████████████████████████████████████████████| 15502/15502 [00:04<00:00, 3184.46it/s]


,tweet,date,Agent,Role,tl,tweetjoin
0,and now i want brimstone in my garden\ni want ...,2021-09-29 22:30:50,Brimstone,Controller,"[want, brimstone, garden, want, rose, set, fire]",want brimstone garden want rose set fire
1,why half of the brimstone players are literall...,2021-09-29 22:14:44,Brimstone,Controller,"[half, brimstone, player, liter, born, brain]",half brimstone player liter born brain
2,@DavidLevy101 @Meloniemaher1 That fucker gets ...,2021-09-29 22:07:50,Brimstone,Controller,"[fucker, get, fire, brimstone, version, hell]",fucker get fire brimstone version hell
3,They also did these cute lil' ghosts! \n\nRobo...,2021-09-29 22:01:21,Brimstone,Controller,"[also, cute, lil, ghost, robot, ghost, fun, co...",also cute lil ghost robot ghost fun concept th...
4,@AuthorDelaneyW @mis_cue @C_Stroop That is not...,2021-09-29 21:57:18,Brimstone,Controller,"[true, fire, brimstone, church, believ, way, c...",true fire brimstone church believ way christia...
...,...,...,...,...,...,...
15497,I'm really confused as to why they did what th...,2021-09-29 07:17:43,Kayo,Initiator,"[im, realli, confus, adam, eric, like, gh0rl, ...",im realli confus adam eric like gh0rl imagin t...
15498,@ChidgR @maxblake44 @Foxtel @kayosports @arara...,2021-09-29 07:15:34,Kayo,Initiator,"[ok, abl, get, kayo, app, smart, tv, use, sams...",ok abl get kayo app smart tv use samsung might...
15499,May interested ba \n\nNCT STICKY VER SEALED\n\...,2021-09-29 07:13:49,Kayo,Initiator,"[may, interest, ba, nct, sticki, ver, seal, 55...",may interest ba nct sticki ver seal 550 50 oct...
15500,@BrokeOnTwice Shout out to all Communities her...,2021-09-29 07:12:59,Kayo,Initiator,"[shout, commun, like, brother, tgg, fam, well,...",shout commun like brother tgg fam well im impr...


In [8]:
X = list(tweets['tweetjoin'].values)
vectorizer = CountVectorizer()
bow = vectorizer.fit_transform(X)

lda = LatentDirichletAllocation(n_components = 5, random_state=0)
lda.fit(bow)

LatentDirichletAllocation(n_components=5, random_state=0)

In [9]:
lda.transform(bow)

array([[0.02548767, 0.02551429, 0.02517225, 0.02522868, 0.89859711],
       [0.02927298, 0.02911317, 0.02907233, 0.88287371, 0.02966781],
       [0.88413729, 0.02894386, 0.02925854, 0.02883949, 0.02882082],
       ...,
       [0.94574463, 0.01341578, 0.01345926, 0.01348915, 0.01389118],
       [0.6065247 , 0.00818781, 0.00827167, 0.00810327, 0.36891255],
       [0.0067801 , 0.30374135, 0.17325531, 0.00675367, 0.50946957]])

In [10]:
lda.score(bow)

-1444159.2426271024

In [13]:
pyLDAvis.enable_notebook()

Source: https://www.kaggle.com/mazenramadan/valorant-agents-tweets?select=valorant_tweets.csv

https://www.youtube.com/watch?v=zi16nl82AMA&list=PLQVvvaa0QuDf2JswnfiGkliBInZnIC4HL&index=11
https://www.codespeedy.com/check-if-a-string-contains-a-special-character-or-not-python/
https://stackoverflow.com/questions/4643142/regex-to-test-if-string-begins-with-http-or-https
https://stackoverflow.com/questions/40035276/adding-arrays-to-dataframe-column
